In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
import shap

In [2]:
df_train = pd.read_parquet("../data/interim/train_final_advanced_features2.parquet")

In [3]:
y = df_train['TARGET']
X = df_train.drop(columns=['TARGET', 'SK_ID_CURR']) #adde sk_id_curr

In [4]:
cat_cols = X.select_dtypes(include=["object"]).columns
num_cols = X.select_dtypes(exclude=["object"]).columns

print(f"Categóricas: {len(cat_cols)}")
print(f"Numéricas: {len(num_cols)}")

Categóricas: 12
Numéricas: 177


In [5]:
for col in X.select_dtypes("object"):
    X[col] = X[col].astype("category")

In [6]:
cat_cols = X.select_dtypes(include=["category"]).columns
num_cols = X.select_dtypes(exclude=["category"]).columns

print(f"Categóricas: {len(cat_cols)}")
print(f"Numéricas: {len(num_cols)}")

Categóricas: 12
Numéricas: 177


In [7]:
print(X.isna().sum().sum())  # must be 0
print(X.shape)

0
(307511, 189)


In [8]:
X_train, X_val, y_train, y_val = train_test_split(
    X,
    y,
    test_size=0.2,
    stratify=y,
    random_state=42)

In [9]:
X_train.shape


(246008, 189)

In [10]:
neg_count = (y_train == 0).sum()
pos_count = (y_train == 1).sum()
optimal_scale = neg_count / pos_count

In [11]:
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',

    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.01,
    'n_estimators': 6000,

    'max_depth': 5,
    'min_child_weight': 15,
    'max_leaves': 31,

    'subsample': 0.75,
    'colsample_bytree': 0.75,
    'colsample_bylevel': 0.8,
    'colsample_bynode': 0.85,

    'scale_pos_weight': optimal_scale,

    'reg_alpha': 0.5,
    'reg_lambda': 2.5,
    'gamma': 1.5,
    'max_delta_step': 2,
}


xgb_model = XGBClassifier(**params, n_jobs=-1, random_state=42,  early_stopping_rounds=300, enable_categorical=True) #added enable_categorical

In [12]:
xgb_model.fit(
    X_train,
    y_train,
    eval_set=[(X_val, y_val)],
    verbose=True)

[0]	validation_0-auc:0.70068
[1]	validation_0-auc:0.71356
[2]	validation_0-auc:0.72454
[3]	validation_0-auc:0.72575
[4]	validation_0-auc:0.72618
[5]	validation_0-auc:0.72808
[6]	validation_0-auc:0.72842
[7]	validation_0-auc:0.72963
[8]	validation_0-auc:0.72943
[9]	validation_0-auc:0.72893
[10]	validation_0-auc:0.73039
[11]	validation_0-auc:0.73179
[12]	validation_0-auc:0.73201
[13]	validation_0-auc:0.73160
[14]	validation_0-auc:0.73260
[15]	validation_0-auc:0.73272
[16]	validation_0-auc:0.73279
[17]	validation_0-auc:0.73364
[18]	validation_0-auc:0.73404
[19]	validation_0-auc:0.73473
[20]	validation_0-auc:0.73494
[21]	validation_0-auc:0.73498
[22]	validation_0-auc:0.73453
[23]	validation_0-auc:0.73427
[24]	validation_0-auc:0.73417
[25]	validation_0-auc:0.73393
[26]	validation_0-auc:0.73450
[27]	validation_0-auc:0.73454
[28]	validation_0-auc:0.73452
[29]	validation_0-auc:0.73524
[30]	validation_0-auc:0.73618
[31]	validation_0-auc:0.73603
[32]	validation_0-auc:0.73590
[33]	validation_0-au

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=0.8, colsample_bynode=0.85,
              colsample_bytree=0.75, device='cuda', early_stopping_rounds=300,
              enable_categorical=True, eval_metric='auc', feature_types=None,
              feature_weights=None, gamma=1.5, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.01, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=2, max_depth=5,
              max_leaves=31, min_child_weight=15, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=6000,
              n_jobs=-1, num_parallel_tree=None, ...)

In [13]:

# ==========================================
# CONFIGURACIÓN 1: ULTRA CONSERVADORA
# Mayor generalización, menos overfit
# ==========================================
params_ultra_conservative = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',

    'tree_method': 'hist',
    'device': 'cuda',

    # Learning muy bajo para aprendizaje gradual
    'learning_rate': 0.005,
    'n_estimators': 10000,

    # Árboles muy simples
    'max_depth': 3,
    'min_child_weight': 30,
    'max_leaves': 15,

    # Sampling muy agresivo para diversidad
    'subsample': 0.6,
    'colsample_bytree': 0.6,
    'colsample_bylevel': 0.6,
    'colsample_bynode': 0.6,

    # Balance de clases
    'scale_pos_weight': optimal_scale,

    # Regularización muy fuerte
    'reg_alpha': 2.0,      # L1
    'reg_lambda': 5.0,     # L2
    'gamma': 3.0,          # Min loss reduction
    'max_delta_step': 1,
}

# ==========================================
# CONFIGURACIÓN 2: CONSERVADORA BALANCEADA
# Buen equilibrio generalización/performance
# ==========================================
params_conservative_balanced = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',

    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.008,
    'n_estimators': 7500,

    'max_depth': 4,
    'min_child_weight': 20,
    'max_leaves': 20,

    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'colsample_bylevel': 0.75,
    'colsample_bynode': 0.8,

    'scale_pos_weight': optimal_scale,

    'reg_alpha': 1.0,
    'reg_lambda': 3.0,
    'gamma': 2.0,
    'max_delta_step': 1,
}

# ==========================================
# CONFIGURACIÓN 3: MODERADA
# Tu configuración actual mejorada
# ==========================================
params_moderate = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',

    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.01,
    'n_estimators': 6000,

    'max_depth': 5,
    'min_child_weight': 15,
    'max_leaves': 31,

    'subsample': 0.75,
    'colsample_bytree': 0.75,
    'colsample_bylevel': 0.8,
    'colsample_bynode': 0.85,

    'scale_pos_weight': optimal_scale,

    'reg_alpha': 0.5,
    'reg_lambda': 2.5,
    'gamma': 1.5,
    'max_delta_step': 2,
}

# ==========================================
# CONFIGURACIÓN 4: AGRESIVA PARA RECALL
# Maximiza detección de defaults
# ==========================================
params_high_recall = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',

    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.015,
    'n_estimators': 5000,

    # Árboles más profundos para capturar patrones complejos
    'max_depth': 6,
    'min_child_weight': 8,
    'max_leaves': 50,

    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'colsample_bylevel': 0.85,
    'colsample_bynode': 0.9,

    # Más peso a clase positiva
    'scale_pos_weight': optimal_scale * 1.3,

    # Regularización más suave
    'reg_alpha': 0.3,
    'reg_lambda': 1.5,
    'gamma': 0.5,
    'max_delta_step': 3,
}

# ==========================================
# CONFIGURACIÓN 5: AGRESIVA PARA PRECISION
# Minimiza falsos positivos
# ==========================================
params_high_precision = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',

    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.012,
    'n_estimators': 4309,

    'max_depth': 5,
    'min_child_weight': 25,
    'max_leaves': 25,

    'subsample': 0.75,
    'colsample_bytree': 0.75,
    'colsample_bylevel': 0.8,
    'colsample_bynode': 0.85,

    # Menos peso a positivos para ser más selectivo
    'scale_pos_weight': optimal_scale * 0.8,

    'reg_alpha': 0.8,
    'reg_lambda': 2.5,
    'gamma': 2.0,
    'max_delta_step': 1,
}

# ==========================================
# CONFIGURACIÓN 6: DART (Dropout Trees)
# Previene overfit con dropout en árboles
# ==========================================
params_dart = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',

    'tree_method': 'hist',
    'device': 'cuda',
    'booster': 'dart',  # Usa DART en lugar de gbtree

    'learning_rate': 0.01,
    'n_estimators': 6000,

    'max_depth': 5,
    'min_child_weight': 15,

    'subsample': 0.8,
    'colsample_bytree': 0.8,

    'scale_pos_weight': optimal_scale,

    # DART específicos
    'sample_type': 'uniform',
    'normalize_type': 'tree',
    'rate_drop': 0.1,
    'skip_drop': 0.5,

    'reg_alpha': 0.5,
    'reg_lambda': 2.0,
    'gamma': 1.0,
}

# ==========================================
# CONFIGURACIÓN 7: FAST & LIGHT
# Rápido para iteraciones, menor memoria
# ==========================================
params_fast = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',

    'tree_method': 'hist',
    'device': 'cuda',

    # Más rápido pero efectivo
    'learning_rate': 0.02,
    'n_estimators': 3000,

    'max_depth': 4,
    'min_child_weight': 20,
    'max_leaves': 15,
    'max_bin': 128,  # Reduce bins para velocidad

    'subsample': 0.7,
    'colsample_bytree': 0.7,

    'scale_pos_weight': optimal_scale,

    'reg_alpha': 1.0,
    'reg_lambda': 2.0,
    'gamma': 1.5,
}

# ==========================================
# CONFIGURACIÓN 8: DEEP TREES
# Árboles profundos con fuerte regularización
# ==========================================
params_deep = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',

    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.008,
    'n_estimators': 7000,

    # Muy profundos
    'max_depth': 8,
    'min_child_weight': 10,
    'max_leaves': 100,

    # Sampling fuerte para compensar profundidad
    'subsample': 0.6,
    'colsample_bytree': 0.6,
    'colsample_bylevel': 0.6,
    'colsample_bynode': 0.6,

    'scale_pos_weight': optimal_scale,

    # Regularización muy fuerte
    'reg_alpha': 3.0,
    'reg_lambda': 5.0,
    'gamma': 4.0,
    'max_delta_step': 1,
}

# ==========================================
# CONFIGURACIÓN 9: MONOTONE CONSTRAINTS
# Fuerza relaciones lógicas en features
# ==========================================
params_monotone = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',

    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.01,
    'n_estimators': 6000,

    'max_depth': 5,
    'min_child_weight': 15,

    'subsample': 0.8,
    'colsample_bytree': 0.8,

    'scale_pos_weight': optimal_scale,

    # Monotone constraints (ajustar según tus features)
    # +1: a mayor valor, mayor probabilidad de default
    # -1: a mayor valor, menor probabilidad de default
    # 0: sin restricción
    # Ejemplo para primeras 10 features (ajustar según tu caso)
    'monotone_constraints': '(0,0,0,0,0,1,1,-1,1,0)',  # Ejemplo

    'reg_alpha': 0.5,
    'reg_lambda': 2.0,
    'gamma': 1.0,
}

# ==========================================
# CONFIGURACIÓN 10: INTERACTION CONSTRAINTS
# Limita qué features pueden interactuar
# ==========================================
params_interaction = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',

    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.01,
    'n_estimators': 6000,

    'max_depth': 6,
    'min_child_weight': 15,

    'subsample': 0.8,
    'colsample_bytree': 0.8,

    'scale_pos_weight': optimal_scale,

    # Permite interacciones solo dentro de grupos
    # Ejemplo: [[0,1,2], [3,4,5]] significa que features 0,1,2 pueden interactuar entre sí
    # y 3,4,5 entre sí, pero no entre grupos
    # 'interaction_constraints': [[0,1,2,3], [4,5,6,7], [8,9,10]],  # Ajustar a tus features

    'reg_alpha': 0.5,
    'reg_lambda': 2.0,
    'gamma': 1.0,
}

# ==========================================
# CONFIGURACIÓN 11: TWO-STAGE ENSEMBLE
# Entrena dos modelos complementarios
# ==========================================
params_ensemble_stage1 = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',

    'tree_method': 'hist',
    'device': 'cuda',

    # Enfocado en generalización
    'learning_rate': 0.008,
    'n_estimators': 6000,

    'max_depth': 4,
    'min_child_weight': 25,

    'subsample': 0.7,
    'colsample_bytree': 0.7,

    'scale_pos_weight': optimal_scale,

    'reg_alpha': 1.5,
    'reg_lambda': 3.0,
    'gamma': 2.0,
}

params_ensemble_stage2 = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',

    'tree_method': 'hist',
    'device': 'cuda',

    # Enfocado en capturar residuos
    'learning_rate': 0.012,
    'n_estimators': 4000,

    'max_depth': 6,
    'min_child_weight': 10,

    'subsample': 0.8,
    'colsample_bytree': 0.8,

    'scale_pos_weight': optimal_scale * 1.2,

    'reg_alpha': 0.3,
    'reg_lambda': 1.5,
    'gamma': 0.8,
}

# =========================================
# BASADO EN EL MEJOR AUC 5
# =========================================

params_best_auc = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.010,  # Ligeramente menor que 0.012
    'n_estimators': 5000,

    'max_depth': 5,
    'min_child_weight': 22,  # Entre 20 y 25
    'max_leaves': 28,

    'subsample': 0.75,
    'colsample_bytree': 0.76,
    'colsample_bylevel': 0.80,
    'colsample_bynode': 0.85,

    'scale_pos_weight': optimal_scale * 0.85,  # Entre 0.8 y 1.0

    'reg_alpha': 0.7,
    'reg_lambda': 2.5,
    'gamma': 1.8,
    'max_delta_step': 1,
}

# ==========================================
# RESUMEN DE CONFIGURACIONES
# ==========================================
configs = {
    '1_ultra_conservative': params_ultra_conservative,
    '2_conservative_balanced': params_conservative_balanced,
    '3_moderate': params_moderate,
    '4_high_recall': params_high_recall,
    '5_high_precision': params_high_precision,
    #'6_dart': params_dart,         #Tarda como 3 horas en entrenar con gpu
    '7_fast': params_fast,
    '8_deep': params_deep,
    '9_monotone': params_monotone,
    '10_interaction': params_interaction,
    '11_ensemble_s1': params_ensemble_stage1,
    '11_ensemble_s2': params_ensemble_stage2,
    '12_best_auc': params_best_auc,
}

# ==========================================
# FUNCIÓN PARA ENTRENAR Y COMPARAR
# ==========================================
def train_and_evaluate(config_name, params, X_train, y_train, X_val, y_val):
    """
    Entrena un modelo con la configuración dada y evalúa
    """
    print(f"\n{'='*70}")
    print(f"Entrenando: {config_name}")
    print(f"{'='*70}")

    # Preparar parámetros
    train_params = params.copy()

    # Extraer n_estimators y early_stopping_rounds
    n_estimators = train_params.pop('n_estimators', 5000)
    early_stopping = min(500, n_estimators // 10)

    # Crear modelo
    model = XGBClassifier(
        **train_params,
        n_estimators=n_estimators,
        n_jobs=-1,
        random_state=42,
        enable_categorical=True, #added here too
        early_stopping_rounds=early_stopping
    )

    # Entrenar
    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        verbose=200
    )

    # Evaluar
    from sklearn.metrics import roc_auc_score, precision_score, recall_score, confusion_matrix

    y_proba = model.predict_proba(X_val)[:, 1]
    y_pred = (y_proba >= 0.5).astype(int)

    auc = roc_auc_score(y_val, y_proba)
    precision = precision_score(y_val, y_pred)
    recall = recall_score(y_val, y_pred)

    tn, fp, fn, tp = confusion_matrix(y_val, y_pred).ravel()
    fpr = fp / (fp + tn)

    print(f"\n{'='*70}")
    print(f"RESULTADOS: {config_name}")
    print(f"{'='*70}")
    print(f"AUC:       {auc:.5f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall:    {recall:.4f}")
    print(f"FPR:       {fpr:.4f}")
    print(f"Best iteration: {model.best_iteration}")

    return {
        'config': config_name,
        'model': model,
        'auc': auc,
        'precision': precision,
        'recall': recall,
        'fpr': fpr,
        'best_iteration': model.best_iteration
    }

In [14]:
results = {}

for i, j in configs.items():
    results[i] = train_and_evaluate(i, j,  X_train, y_train, X_val, y_val)


Entrenando: 1_ultra_conservative
[0]	validation_0-auc:0.68263
[200]	validation_0-auc:0.73667
[400]	validation_0-auc:0.74403
[600]	validation_0-auc:0.74984
[800]	validation_0-auc:0.75477
[1000]	validation_0-auc:0.75885
[1200]	validation_0-auc:0.76209
[1400]	validation_0-auc:0.76466
[1600]	validation_0-auc:0.76677
[1800]	validation_0-auc:0.76842
[2000]	validation_0-auc:0.76984
[2200]	validation_0-auc:0.77103
[2400]	validation_0-auc:0.77205
[2600]	validation_0-auc:0.77298
[2800]	validation_0-auc:0.77376
[3000]	validation_0-auc:0.77453
[3200]	validation_0-auc:0.77520
[3400]	validation_0-auc:0.77582
[3600]	validation_0-auc:0.77635
[3800]	validation_0-auc:0.77680
[4000]	validation_0-auc:0.77724
[4200]	validation_0-auc:0.77769
[4400]	validation_0-auc:0.77804
[4600]	validation_0-auc:0.77841
[4800]	validation_0-auc:0.77878
[5000]	validation_0-auc:0.77905
[5200]	validation_0-auc:0.77936
[5400]	validation_0-auc:0.77963
[5600]	validation_0-auc:0.77991
[5800]	validation_0-auc:0.78023
[6000]	valida

C:\Users\usuario\Desktop\4geeks\Final_Proyect_Credit_Default_Risk-main\.venv\Lib\site-packages\xgboost\core.py:774: UserWarning: [21:08:03] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:62: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  return func(**kwargs)



RESULTADOS: 1_ultra_conservative
AUC:       0.78328
Precision: 0.1844
Recall:    0.6935
FPR:       0.2694
Best iteration: 9989

Entrenando: 2_conservative_balanced
[0]	validation_0-auc:0.69406
[200]	validation_0-auc:0.74263
[400]	validation_0-auc:0.75638
[600]	validation_0-auc:0.76462
[800]	validation_0-auc:0.76931
[1000]	validation_0-auc:0.77238
[1200]	validation_0-auc:0.77452
[1400]	validation_0-auc:0.77615
[1600]	validation_0-auc:0.77745
[1800]	validation_0-auc:0.77862
[2000]	validation_0-auc:0.77954
[2200]	validation_0-auc:0.78025
[2400]	validation_0-auc:0.78093
[2600]	validation_0-auc:0.78142
[2800]	validation_0-auc:0.78198
[3000]	validation_0-auc:0.78237
[3200]	validation_0-auc:0.78276
[3400]	validation_0-auc:0.78304
[3600]	validation_0-auc:0.78330
[3800]	validation_0-auc:0.78355
[4000]	validation_0-auc:0.78374
[4200]	validation_0-auc:0.78388
[4400]	validation_0-auc:0.78402
[4600]	validation_0-auc:0.78416
[4800]	validation_0-auc:0.78428
[5000]	validation_0-auc:0.78442
[5200]	val

In [15]:
for key, d in results.items():
    print(key)
    print({k: d[k] for k in ['auc', 'precision', 'recall', 'fpr', 'best_iteration']})

1_ultra_conservative
{'auc': 0.7832805424166056, 'precision': 0.1843641231593039, 'recall': 0.6934541792547835, 'fpr': 0.26941172308889594, 'best_iteration': 9989}
2_conservative_balanced
{'auc': 0.7852021528035383, 'precision': 0.19520668943587327, 'recall': 0.6676737160120846, 'fpr': 0.24173122501680286, 'best_iteration': 7423}
3_moderate
{'auc': 0.7852313785732146, 'precision': 0.20209410874227324, 'recall': 0.6453172205438067, 'fpr': 0.22374332307474618, 'best_iteration': 4437}
4_high_recall
{'auc': 0.7820181897286097, 'precision': 0.17855891042010916, 'recall': 0.7182275931520644, 'fpr': 0.29015883122855424, 'best_iteration': 1499}
5_high_precision
{'auc': 0.7857739415927055, 'precision': 0.2232212666145426, 'recall': 0.5750251762336355, 'fpr': 0.17572252290494889, 'best_iteration': 4306}
7_fast
{'auc': 0.7844048528599699, 'precision': 0.1951104287232786, 'recall': 0.6654582074521651, 'fpr': 0.24107679790583325, 'best_iteration': 2996}
8_deep
{'auc': 0.7844704736900922, 'precision

In [16]:
# ==========================================
# MICRO-OPTIMIZACIÓN DE 5_HIGH_PRECISION
# Exploración fina alrededor del mejor modelo
# ==========================================

# BASE DE REFERENCIA (5_high_precision - AUC: 0.78817)
# 'learning_rate': 0.012
# 'scale_pos_weight': optimal_scale * 0.80
# 'min_child_weight': 25
# 'max_leaves': 25
# 'reg_alpha': 0.8, 'reg_lambda': 2.5, 'gamma': 2.0

# ==========================================
# GRUPO 1: AJUSTE DE SCALE_POS_WEIGHT
# El hiperparámetro MÁS crítico según resultados
# ==========================================

# Config 13: Scale más bajo (más conservador)
params_13_scale_low = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.012,
    'n_estimators': 6000,

    'max_depth': 5,
    'min_child_weight': 25,
    'max_leaves': 25,

    'subsample': 0.75,
    'colsample_bytree': 0.75,
    'colsample_bylevel': 0.80,
    'colsample_bynode': 0.85,

    'scale_pos_weight': optimal_scale * 0.76,  # Entre 0.76-0.80

    'reg_alpha': 0.8,
    'reg_lambda': 2.5,
    'gamma': 2.0,
    'max_delta_step': 1,
}

# Config 14: Scale ligeramente más alto
params_14_scale_med = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.012,
    'n_estimators': 6000,

    'max_depth': 5,
    'min_child_weight': 25,
    'max_leaves': 25,

    'subsample': 0.75,
    'colsample_bytree': 0.75,
    'colsample_bylevel': 0.80,
    'colsample_bynode': 0.85,

    'scale_pos_weight': optimal_scale * 0.82,  # Entre 0.80-0.85

    'reg_alpha': 0.8,
    'reg_lambda': 2.5,
    'gamma': 2.0,
    'max_delta_step': 1,
}

# ==========================================
# GRUPO 2: AJUSTE DE MIN_CHILD_WEIGHT
# Segundo parámetro más sensible
# ==========================================

# Config 15: Más restrictivo (menos overfit)
params_15_mcw_high = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.012,
    'n_estimators': 6000,

    'max_depth': 5,
    'min_child_weight': 28,  # Más alto que 25
    'max_leaves': 25,

    'subsample': 0.75,
    'colsample_bytree': 0.75,
    'colsample_bylevel': 0.80,
    'colsample_bynode': 0.85,

    'scale_pos_weight': optimal_scale * 0.80,

    'reg_alpha': 0.8,
    'reg_lambda': 2.5,
    'gamma': 2.0,
    'max_delta_step': 1,
}

# Config 16: Menos restrictivo (más capacidad)
params_16_mcw_low = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.012,
    'n_estimators': 6000,

    'max_depth': 5,
    'min_child_weight': 22,  # Más bajo que 25
    'max_leaves': 25,

    'subsample': 0.75,
    'colsample_bytree': 0.75,
    'colsample_bylevel': 0.80,
    'colsample_bynode': 0.85,

    'scale_pos_weight': optimal_scale * 0.80,

    'reg_alpha': 0.8,
    'reg_lambda': 2.5,
    'gamma': 2.0,
    'max_delta_step': 1,
}

# ==========================================
# GRUPO 3: AJUSTE DE REGULARIZACIÓN (L1, L2, GAMMA)
# Balance entre los 3 tipos de regularización
# ==========================================

# Config 17: Regularización más suave
params_17_reg_soft = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.012,
    'n_estimators': 6000,

    'max_depth': 5,
    'min_child_weight': 25,
    'max_leaves': 25,

    'subsample': 0.75,
    'colsample_bytree': 0.75,
    'colsample_bylevel': 0.80,
    'colsample_bynode': 0.85,

    'scale_pos_weight': optimal_scale * 0.80,

    'reg_alpha': 0.6,   # Menos L1
    'reg_lambda': 2.2,  # Menos L2
    'gamma': 1.7,       # Menos gamma
    'max_delta_step': 1,
}

# Config 18: Regularización más fuerte
params_18_reg_strong = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.012,
    'n_estimators': 6000,

    'max_depth': 5,
    'min_child_weight': 25,
    'max_leaves': 25,

    'subsample': 0.75,
    'colsample_bytree': 0.75,
    'colsample_bylevel': 0.80,
    'colsample_bynode': 0.85,

    'scale_pos_weight': optimal_scale * 0.80,

    'reg_alpha': 1.0,   # Más L1
    'reg_lambda': 2.8,  # Más L2
    'gamma': 2.3,       # Más gamma
    'max_delta_step': 1,
}

# Config 19: Balance L1 vs L2 diferente
params_19_reg_balance = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.012,
    'n_estimators': 6000,

    'max_depth': 5,
    'min_child_weight': 25,
    'max_leaves': 25,

    'subsample': 0.75,
    'colsample_bytree': 0.75,
    'colsample_bylevel': 0.80,
    'colsample_bynode': 0.85,

    'scale_pos_weight': optimal_scale * 0.80,

    'reg_alpha': 1.0,   # Más L1 (feature selection)
    'reg_lambda': 2.0,  # Menos L2 (suavidad)
    'gamma': 2.2,
    'max_delta_step': 1,
}

# ==========================================
# GRUPO 4: LEARNING RATE + MAX_LEAVES
# Combinaciones ajustadas
# ==========================================

# Config 20: Learning rate más lento, más árboles simples
params_20_slow_simple = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.010,  # Más lento
    'n_estimators': 7200,    # Más iteraciones

    'max_depth': 5,
    'min_child_weight': 25,
    'max_leaves': 22,  # Más simple

    'subsample': 0.75,
    'colsample_bytree': 0.75,
    'colsample_bylevel': 0.80,
    'colsample_bynode': 0.85,

    'scale_pos_weight': optimal_scale * 0.80,

    'reg_alpha': 0.8,
    'reg_lambda': 2.5,
    'gamma': 2.0,
    'max_delta_step': 1,
}

# Config 21: Learning rate más rápido, árboles ligeramente más complejos
params_21_fast_complex = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.013,  # Más rápido
    'n_estimators': 5500,    # Menos iteraciones

    'max_depth': 5,
    'min_child_weight': 25,
    'max_leaves': 28,  # Más complejo

    'subsample': 0.75,
    'colsample_bytree': 0.75,
    'colsample_bylevel': 0.80,
    'colsample_bynode': 0.85,

    'scale_pos_weight': optimal_scale * 0.80,

    'reg_alpha': 0.8,
    'reg_lambda': 2.5,
    'gamma': 2.0,
    'max_delta_step': 1,
}

# ==========================================
# GRUPO 5: SAMPLING VARIATIONS
# Ajustes en subsample y colsample_*
# ==========================================

# Config 22: Más sampling (más diversidad)
params_22_high_sample = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.012,
    'n_estimators': 6000,

    'max_depth': 5,
    'min_child_weight': 25,
    'max_leaves': 25,

    'subsample': 0.78,           # Más alto
    'colsample_bytree': 0.78,    # Más alto
    'colsample_bylevel': 0.82,   # Más alto
    'colsample_bynode': 0.87,    # Más alto

    'scale_pos_weight': optimal_scale * 0.80,

    'reg_alpha': 0.8,
    'reg_lambda': 2.5,
    'gamma': 2.0,
    'max_delta_step': 1,
}

# Config 23: Menos sampling (más conservador)
params_23_low_sample = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.012,
    'n_estimators': 6000,

    'max_depth': 5,
    'min_child_weight': 25,
    'max_leaves': 25,

    'subsample': 0.72,           # Más bajo
    'colsample_bytree': 0.72,    # Más bajo
    'colsample_bylevel': 0.78,   # Más bajo
    'colsample_bynode': 0.83,    # Más bajo

    'scale_pos_weight': optimal_scale * 0.80,

    'reg_alpha': 0.8,
    'reg_lambda': 2.5,
    'gamma': 2.0,
    'max_delta_step': 1,
}

# ==========================================
# GRUPO 6: COMBINACIONES ÓPTIMAS
# Mejores combinaciones basadas en intuición
# ==========================================

# Config 24: Combinación conservadora óptima
params_24_ultra_optimal = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.011,
    'n_estimators': 6500,

    'max_depth': 5,
    'min_child_weight': 26,  # Ligeramente más restrictivo
    'max_leaves': 24,        # Ligeramente más simple

    'subsample': 0.76,
    'colsample_bytree': 0.76,
    'colsample_bylevel': 0.81,
    'colsample_bynode': 0.86,

    'scale_pos_weight': optimal_scale * 0.78,  # Entre 0.76 y 0.80

    'reg_alpha': 0.85,  # Entre 0.8 y 0.9
    'reg_lambda': 2.6,  # Entre 2.5 y 2.7
    'gamma': 2.1,       # Entre 2.0 y 2.2
    'max_delta_step': 1,
}

# Config 25: Combinación agresiva óptima
params_25_balanced_optimal = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.0125,  # Entre 0.012 y 0.013
    'n_estimators': 5800,

    'max_depth': 5,
    'min_child_weight': 24,  # Ligeramente menos restrictivo
    'max_leaves': 26,        # Ligeramente más complejo

    'subsample': 0.77,
    'colsample_bytree': 0.77,
    'colsample_bylevel': 0.81,
    'colsample_bynode': 0.86,

    'scale_pos_weight': optimal_scale * 0.81,  # Entre 0.80 y 0.82

    'reg_alpha': 0.75,  # Menos L1
    'reg_lambda': 2.4,  # Menos L2
    'gamma': 1.9,       # Menos gamma
    'max_delta_step': 1,
}

# ==========================================
# DICCIONARIO DE TODAS LAS CONFIGS
# ==========================================
all_fine_tuned_configs = {
    '13_scale_low': params_13_scale_low,
    '14_scale_med': params_14_scale_med,
    '15_mcw_high': params_15_mcw_high,
    '16_mcw_low': params_16_mcw_low,
    '17_reg_soft': params_17_reg_soft,
    '18_reg_strong': params_18_reg_strong,
    '19_reg_balance': params_19_reg_balance,
    '20_slow_simple': params_20_slow_simple,
    '21_fast_complex': params_21_fast_complex,
    '22_high_sample': params_22_high_sample,
    '23_low_sample': params_23_low_sample,
    '24_ultra_optimal': params_24_ultra_optimal,
    '25_balanced_optimal': params_25_balanced_optimal,
}



In [17]:
results_tuned = {}

for i, j in all_fine_tuned_configs.items():
    results_tuned[i] = train_and_evaluate(i, j,  X_train, y_train, X_val, y_val)


Entrenando: 13_scale_low
[0]	validation_0-auc:0.69879
[200]	validation_0-auc:0.75496
[400]	validation_0-auc:0.76844
[600]	validation_0-auc:0.77429
[800]	validation_0-auc:0.77745
[1000]	validation_0-auc:0.77947
[1200]	validation_0-auc:0.78094
[1400]	validation_0-auc:0.78223
[1600]	validation_0-auc:0.78323
[1800]	validation_0-auc:0.78377
[2000]	validation_0-auc:0.78425
[2200]	validation_0-auc:0.78466
[2400]	validation_0-auc:0.78489
[2600]	validation_0-auc:0.78519
[2800]	validation_0-auc:0.78538
[3000]	validation_0-auc:0.78552
[3200]	validation_0-auc:0.78576
[3400]	validation_0-auc:0.78588
[3600]	validation_0-auc:0.78579
[3800]	validation_0-auc:0.78577
[3858]	validation_0-auc:0.78575

RESULTADOS: 13_scale_low
AUC:       0.78592
Precision: 0.2262
Recall:    0.5688
FPR:       0.1709
Best iteration: 3358

Entrenando: 14_scale_med
[0]	validation_0-auc:0.69805
[200]	validation_0-auc:0.75516
[400]	validation_0-auc:0.76823
[600]	validation_0-auc:0.77426
[800]	validation_0-auc:0.77722
[1000]	val

In [18]:
for i, j in results_tuned.items():
    print(i)
    print({k: j[k] for k in ['auc', 'precision', 'recall', 'fpr', 'best_iteration']})

13_scale_low
{'auc': 0.7859237966910972, 'precision': 0.22620954822172382, 'recall': 0.5687814702920443, 'fpr': 0.17085853762071526, 'best_iteration': 3358}
14_scale_med
{'auc': 0.7852147297879168, 'precision': 0.21849051008303677, 'recall': 0.5935548841893252, 'fpr': 0.18644097774947824, 'best_iteration': 3485}
15_mcw_high
{'auc': 0.7857063739216362, 'precision': 0.22142584826534503, 'recall': 0.5848942598187311, 'fpr': 0.1806041954083979, 'best_iteration': 3693}
16_mcw_low
{'auc': 0.7856371034326848, 'precision': 0.22231634067457262, 'recall': 0.5814702920443102, 'fpr': 0.17862322685627366, 'best_iteration': 3791}
17_reg_soft
{'auc': 0.7855525877363554, 'precision': 0.22206714739861982, 'recall': 0.5768378650553877, 'fpr': 0.1774558703880576, 'best_iteration': 4021}
18_reg_strong
{'auc': 0.785848600182173, 'precision': 0.22382925494488312, 'recall': 0.5766364551863041, 'fpr': 0.1755987123704411, 'best_iteration': 4245}
19_reg_balance
{'auc': 0.7851573344943844, 'precision': 0.2202666

In [19]:
xgb_model = XGBClassifier(**params_best_auc, n_jobs=-1, random_state=42, enable_categorical=True)
xgb_model.fit(
    X_train,
    y_train,
    eval_set=[(X_val, y_val)],
    verbose=True)

[0]	validation_0-auc:0.69642
[1]	validation_0-auc:0.70345
[2]	validation_0-auc:0.72175
[3]	validation_0-auc:0.72533
[4]	validation_0-auc:0.72769
[5]	validation_0-auc:0.73044
[6]	validation_0-auc:0.73001
[7]	validation_0-auc:0.73105
[8]	validation_0-auc:0.73016
[9]	validation_0-auc:0.72978
[10]	validation_0-auc:0.73152
[11]	validation_0-auc:0.73250
[12]	validation_0-auc:0.73305
[13]	validation_0-auc:0.73285
[14]	validation_0-auc:0.73356
[15]	validation_0-auc:0.73369
[16]	validation_0-auc:0.73357
[17]	validation_0-auc:0.73392
[18]	validation_0-auc:0.73428
[19]	validation_0-auc:0.73454
[20]	validation_0-auc:0.73450
[21]	validation_0-auc:0.73457
[22]	validation_0-auc:0.73440
[23]	validation_0-auc:0.73442
[24]	validation_0-auc:0.73461
[25]	validation_0-auc:0.73424
[26]	validation_0-auc:0.73488
[27]	validation_0-auc:0.73495
[28]	validation_0-auc:0.73460
[29]	validation_0-auc:0.73562
[30]	validation_0-auc:0.73660
[31]	validation_0-auc:0.73656
[32]	validation_0-auc:0.73644
[33]	validation_0-au

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=0.8, colsample_bynode=0.85,
              colsample_bytree=0.76, device='cuda', early_stopping_rounds=None,
              enable_categorical=True, eval_metric='auc', feature_types=None,
              feature_weights=None, gamma=1.8, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.01, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=1, max_depth=5,
              max_leaves=28, min_child_weight=22, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=5000,
              n_jobs=-1, num_parallel_tree=None, ...)

In [20]:
#optimal f1 threshold
from sklearn.metrics import f1_score
import numpy as np

y_val_proba = xgb_model.predict_proba(X_val)[:, 1]

thresholds = np.linspace(0.05, 0.6, 100)
f1_scores = [f1_score(y_val, y_val_proba >= t) for t in thresholds]

optimal_threshold = thresholds[np.argmax(f1_scores)]

print(f"Optimal Threshold (F1): {optimal_threshold:.3f}")

Optimal Threshold (F1): 0.600


In [21]:
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix, roc_curve

y_val_pred = xgb_model.predict_proba(X_val)[:, 1]
auc = roc_auc_score(y_val, y_val_pred)

adj_threshold = optimal_threshold

y_val_pred = (y_val_pred >= adj_threshold).astype(int)

report = classification_report(y_val, y_val_pred)

#Confusion matrix manual
TP = np.sum((y_val == 1) & (y_val_pred == 1))
FP = np.sum((y_val == 0) & (y_val_pred == 1))
TN = np.sum((y_val == 0) & (y_val_pred == 0))
FN = np.sum((y_val == 1) & (y_val_pred == 0))

FPR = FP / (FP + TN)
Precision = TP / (TP + FP)

print(f"FPR: {FPR:.4f}")
print(f"Precision (PPV): {Precision:.4f}")
print(f"AUC validación: {auc:.5f}")
print(report)

FPR: 0.1077
Precision (PPV): 0.2720
AUC validación: 0.78548
              precision    recall  f1-score   support

           0       0.95      0.89      0.92     56538
           1       0.27      0.46      0.34      4965

    accuracy                           0.86     61503
   macro avg       0.61      0.68      0.63     61503
weighted avg       0.89      0.86      0.87     61503



In [22]:
# ==========================================
# CONFIGURACIONES XGBOOST PARA 363 FEATURES
# Optimizadas para dataset con feature engineering avanzado
# ==========================================

# IMPORTANTE: Con 363 features (vs ~120 originales), necesitas:
# 1. Más regularización (para evitar overfit con tantas features)
# 2. Más colsample (sampling de features más agresivo)
# 3. Árboles potencialmente más profundos (para capturar interacciones)

# ==========================================
# CONFIG 26: HIGH PRECISION CON MUCHAS FEATURES
# Adaptación del mejor modelo (5_high_precision) para 363 features
# ==========================================
params_26_hp_many_features = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.012,
    'n_estimators': 6000,

    'max_depth': 6,  # +1 vs original (más features = más profundidad OK)
    'min_child_weight': 30,  # +5 (más conservador con muchas features)
    'max_leaves': 30,  # +5

    # ⚡ CRÍTICO: Sampling más agresivo con 363 features
    'subsample': 0.70,  # -0.05
    'colsample_bytree': 0.65,  # -0.10 (muy importante!)
    'colsample_bylevel': 0.70,  # -0.10
    'colsample_bynode': 0.75,  # -0.10

    'scale_pos_weight': optimal_scale * 0.80,

    # Regularización más fuerte
    'reg_alpha': 1.2,   # +0.4
    'reg_lambda': 3.5,  # +1.0
    'gamma': 2.5,       # +0.5
    'max_delta_step': 1,
}

# ==========================================
# CONFIG 27: ULTRA REGULARIZADO PARA MUCHAS FEATURES
# Previene overfit agresivamente
# ==========================================
params_27_ultra_reg = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.010,
    'n_estimators': 7000,

    'max_depth': 5,
    'min_child_weight': 35,
    'max_leaves': 25,

    # Feature sampling MUY agresivo
    'subsample': 0.65,
    'colsample_bytree': 0.60,
    'colsample_bylevel': 0.65,
    'colsample_bynode': 0.70,

    'scale_pos_weight': optimal_scale * 0.80,

    # Regularización extrema
    'reg_alpha': 2.0,
    'reg_lambda': 5.0,
    'gamma': 3.0,
    'max_delta_step': 1,
}

# ==========================================
# CONFIG 28: DEEP TREES CON FEATURE SELECTION
# Árboles profundos + sampling fuerte = explora interacciones
# ==========================================
params_28_deep_selection = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.008,
    'n_estimators': 8000,

    # Profundo para capturar interacciones complejas
    'max_depth': 7,
    'min_child_weight': 25,
    'max_leaves': 50,

    # Sampling agresivo compensa profundidad
    'subsample': 0.65,
    'colsample_bytree': 0.55,  # Muy bajo: cada árbol ve ~200 features
    'colsample_bylevel': 0.60,
    'colsample_bynode': 0.65,

    'scale_pos_weight': optimal_scale * 0.80,

    'reg_alpha': 2.5,  # L1 alto para feature selection
    'reg_lambda': 4.0,
    'gamma': 3.5,
    'max_delta_step': 1,
}

# ==========================================
# CONFIG 29: BALANCED DIVERSITY
# Balance entre exploración y explotación
# ==========================================
params_29_balanced_diverse = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.011,
    'n_estimators': 6500,

    'max_depth': 6,
    'min_child_weight': 28,
    'max_leaves': 35,

    'subsample': 0.68,
    'colsample_bytree': 0.62,
    'colsample_bylevel': 0.68,
    'colsample_bynode': 0.73,

    'scale_pos_weight': optimal_scale * 0.80,

    'reg_alpha': 1.5,
    'reg_lambda': 3.8,
    'gamma': 2.7,
    'max_delta_step': 1,
}

# ==========================================
# CONFIG 30: FEATURE SAMPLING EXTREMO
# Cada árbol ve solo ~30% de features (ensemble diverso)
# ==========================================
params_30_extreme_sampling = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.013,
    'n_estimators': 6000,

    'max_depth': 5,
    'min_child_weight': 25,
    'max_leaves': 28,

    # SAMPLING EXTREMO
    'subsample': 0.60,
    'colsample_bytree': 0.50,  # Solo 180 features por árbol
    'colsample_bylevel': 0.55,
    'colsample_bynode': 0.60,

    'scale_pos_weight': optimal_scale * 0.80,

    'reg_alpha': 1.8,
    'reg_lambda': 3.0,
    'gamma': 2.2,
    'max_delta_step': 1,
}

# ==========================================
# CONFIG 31: L1 DOMINANT (Feature Selection)
# Alta regularización L1 para selección automática de features
# ==========================================
params_31_l1_dominant = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.012,
    'n_estimators': 6000,

    'max_depth': 6,
    'min_child_weight': 30,
    'max_leaves': 32,

    'subsample': 0.70,
    'colsample_bytree': 0.65,
    'colsample_bylevel': 0.70,
    'colsample_bynode': 0.75,

    'scale_pos_weight': optimal_scale * 0.80,

    # L1 muy alto para feature selection
    'reg_alpha': 3.0,  # ⚡ Muy alto
    'reg_lambda': 2.0,  # L2 más bajo
    'gamma': 2.0,
    'max_delta_step': 1,
}

# ==========================================
# CONFIG 32: ADAPTIVE LEARNING
# Learning rate más alto con más regularización
# ==========================================
params_32_adaptive = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.015,  # Más rápido
    'n_estimators': 5000,

    'max_depth': 5,
    'min_child_weight': 32,
    'max_leaves': 27,

    'subsample': 0.68,
    'colsample_bytree': 0.63,
    'colsample_bylevel': 0.68,
    'colsample_bynode': 0.73,

    'scale_pos_weight': optimal_scale * 0.80,

    'reg_alpha': 1.3,
    'reg_lambda': 3.3,
    'gamma': 2.5,
    'max_delta_step': 1,
}

# ==========================================
# CONFIG 33: MODERATE DEPTH + HIGH REG
# Balance clásico pero adaptado
# ==========================================
params_33_moderate_highreg = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.011,
    'n_estimators': 6500,

    'max_depth': 6,
    'min_child_weight': 27,
    'max_leaves': 33,

    'subsample': 0.72,
    'colsample_bytree': 0.67,
    'colsample_bylevel': 0.72,
    'colsample_bynode': 0.77,

    'scale_pos_weight': optimal_scale * 0.80,

    'reg_alpha': 1.4,
    'reg_lambda': 3.6,
    'gamma': 2.6,
    'max_delta_step': 1,
}

# ==========================================
# CONFIG 34: CONSERVATIVE MANY FEATURES
# Muy conservador específicamente para 363 features
# ==========================================
params_34_conservative_363 = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.009,
    'n_estimators': 7500,

    'max_depth': 4,  # Shallow
    'min_child_weight': 40,  # Muy restrictivo
    'max_leaves': 20,

    'subsample': 0.65,
    'colsample_bytree': 0.58,
    'colsample_bylevel': 0.63,
    'colsample_bynode': 0.68,

    'scale_pos_weight': optimal_scale * 0.80,

    'reg_alpha': 2.2,
    'reg_lambda': 4.5,
    'gamma': 3.2,
    'max_delta_step': 1,
}

# ==========================================
# CONFIG 35: FOCUS ON NEW FEATURES
# Optimizado para aprovechar las nuevas features avanzadas
# ==========================================
params_35_new_features_focus = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.012,
    'n_estimators': 6000,

    'max_depth': 6,
    'min_child_weight': 26,
    'max_leaves': 35,

    # Permite más exploración de features
    'subsample': 0.72,
    'colsample_bytree': 0.68,
    'colsample_bylevel': 0.73,
    'colsample_bynode': 0.78,

    'scale_pos_weight': optimal_scale * 0.78,  # Ligeramente ajustado

    'reg_alpha': 1.3,
    'reg_lambda': 3.2,
    'gamma': 2.3,
    'max_delta_step': 1,
}

# ==========================================
# DICCIONARIO COMPLETO
# ==========================================
all_advanced_configs = {
    '26_hp_many_features': params_26_hp_many_features,
    '27_ultra_reg': params_27_ultra_reg,
    '28_deep_selection': params_28_deep_selection,
    '29_balanced_diverse': params_29_balanced_diverse,
    '30_extreme_sampling': params_30_extreme_sampling,
    '31_l1_dominant': params_31_l1_dominant,
    '32_adaptive': params_32_adaptive,
    '33_moderate_highreg': params_33_moderate_highreg,
    '34_conservative_363': params_34_conservative_363,
    '35_new_features_focus': params_35_new_features_focus,
}


In [23]:
results_advanced = {}

for i, j in all_advanced_configs.items():
    results_advanced[i] = train_and_evaluate(i, j,  X_train, y_train, X_val, y_val)


Entrenando: 26_hp_many_features
[0]	validation_0-auc:0.69647
[200]	validation_0-auc:0.75740
[400]	validation_0-auc:0.76887
[600]	validation_0-auc:0.77432
[800]	validation_0-auc:0.77762
[1000]	validation_0-auc:0.77963
[1200]	validation_0-auc:0.78121
[1400]	validation_0-auc:0.78225
[1600]	validation_0-auc:0.78320
[1800]	validation_0-auc:0.78372
[2000]	validation_0-auc:0.78418
[2200]	validation_0-auc:0.78450
[2400]	validation_0-auc:0.78474
[2600]	validation_0-auc:0.78500
[2800]	validation_0-auc:0.78510
[3000]	validation_0-auc:0.78512
[3200]	validation_0-auc:0.78538
[3400]	validation_0-auc:0.78537
[3600]	validation_0-auc:0.78550
[3800]	validation_0-auc:0.78544
[4000]	validation_0-auc:0.78547
[4192]	validation_0-auc:0.78548

RESULTADOS: 26_hp_many_features
AUC:       0.78559
Precision: 0.2232
Recall:    0.5742
FPR:       0.1755
Best iteration: 3692

Entrenando: 27_ultra_reg
[0]	validation_0-auc:0.70115
[200]	validation_0-auc:0.75522
[400]	validation_0-auc:0.76605
[600]	validation_0-auc:0.7

In [24]:
for i, j in results_advanced.items():
    print(i)
    print({k: j[k] for k in ['auc', 'precision', 'recall', 'fpr', 'best_iteration']})

26_hp_many_features
{'auc': 0.785585765967204, 'precision': 0.22324015347271162, 'recall': 0.5742195367573011, 'fpr': 0.17545721461671795, 'best_iteration': 3692}
27_ultra_reg
{'auc': 0.7863216647203599, 'precision': 0.22201918684330746, 'recall': 0.5873111782477342, 'fpr': 0.18072800594290567, 'best_iteration': 4877}
28_deep_selection
{'auc': 0.7865242270195375, 'precision': 0.22628780061909676, 'recall': 0.5742195367573011, 'fpr': 0.17241501291167002, 'best_iteration': 3547}
29_balanced_diverse
{'auc': 0.7858599125214718, 'precision': 0.22443694577415052, 'recall': 0.5760322255790534, 'fpr': 0.17480278750574835, 'best_iteration': 3437}
30_extreme_sampling
{'auc': 0.7860602732694962, 'precision': 0.22125912408759124, 'recall': 0.5861027190332326, 'fpr': 0.18115249920407514, 'best_iteration': 3691}
31_l1_dominant
{'auc': 0.7854490222102668, 'precision': 0.22401811509330835, 'recall': 0.5778449144008057, 'fpr': 0.17577558456259507, 'best_iteration': 3442}
32_adaptive
{'auc': 0.785654297

In [ ]:
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import numpy as np

cat_features = [
    col for col in X.columns
    if X[col].dtype == 'category'
]

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
aucs = []

for fold, (tr, va) in enumerate(skf.split(X, y), 1):

    model = CatBoostClassifier(
        iterations=4000,
        learning_rate=0.03,
        depth=6,
        l2_leaf_reg=3,
        loss_function='Logloss',
        eval_metric='AUC',
        random_seed=42,
        early_stopping_rounds=300,
        verbose=200,
    )

    model.fit(
        X.iloc[tr], y.iloc[tr],
        eval_set=(X.iloc[va], y.iloc[va]),
        cat_features=cat_features
    )

    preds = model.predict_proba(X.iloc[va])[:, 1]
    auc = roc_auc_score(y.iloc[va], preds)
    aucs.append(auc)

    print(f'Fold {fold} AUC: {auc:.5f}')

print(f'\nAUC medio: {np.mean(aucs):.5f} ± {np.std(aucs):.5f}')



0:	test: 0.6352290	best: 0.6352290 (0)	total: 680ms	remaining: 45m 19s
200:	test: 0.7623600	best: 0.7623600 (200)	total: 43.9s	remaining: 13m 49s
400:	test: 0.7699773	best: 0.7699773 (400)	total: 1m 23s	remaining: 12m 32s
600:	test: 0.7739966	best: 0.7739966 (600)	total: 2m 3s	remaining: 11m 40s
800:	test: 0.7761146	best: 0.7761146 (800)	total: 2m 53s	remaining: 11m 34s
1000:	test: 0.7774654	best: 0.7774694 (993)	total: 3m 37s	remaining: 10m 51s
1200:	test: 0.7785639	best: 0.7785639 (1200)	total: 4m 20s	remaining: 10m 6s
1400:	test: 0.7793203	best: 0.7793238 (1398)	total: 5m 3s	remaining: 9m 22s
1600:	test: 0.7799003	best: 0.7799093 (1596)	total: 5m 44s	remaining: 8m 36s
1800:	test: 0.7802872	best: 0.7802872 (1800)	total: 6m 26s	remaining: 7m 51s
2000:	test: 0.7806409	best: 0.7806409 (2000)	total: 7m 7s	remaining: 7m 7s
2200:	test: 0.7808062	best: 0.7808386 (2195)	total: 7m 49s	remaining: 6m 23s
2400:	test: 0.7809957	best: 0.7809957 (2400)	total: 8m 30s	remaining: 5m 40s
2600:	test: 0.

In [35]:
print(aucs)

[0.7816351982003422, 0.7900056715882002, 0.7815013529893292, 0.789682262278456, 0.7824631521772024]


[0.7816351982003422, 0.7900056715882002, 0.7815013529893292, 0.789682262278456, 0.7824631521772024]

In [36]:
from sklearn.metrics import roc_auc_score, classification_report
import numpy as np

# Probabilidades (idéntico conceptualmente)
y_val_proba = model.predict_proba(X_val)[:, 1]

# AUC (esto es lo importante)
auc = roc_auc_score(y_val, y_val_proba)

#Threshold (solo para métricas de clasificación)
adj_threshold = 0.6

y_val_pred = (y_val_proba >= adj_threshold).astype(int)

# Classification report
report = classification_report(y_val, y_val_pred)

#Confusion matrix manual
TP = np.sum((y_val == 1) & (y_val_pred == 1))
FP = np.sum((y_val == 0) & (y_val_pred == 1))
TN = np.sum((y_val == 0) & (y_val_pred == 0))
FN = np.sum((y_val == 1) & (y_val_pred == 0))

FPR = FP / (FP + TN)
Precision = TP / (TP + FP)

print(f"FPR: {FPR:.4f}")
print(f"Precision (PPV): {Precision:.4f}")
print(f"AUC validación: {auc:.5f}")
print(report)


FPR: 0.0004
Precision (PPV): 0.9432
AUC validación: 0.84463
              precision    recall  f1-score   support

           0       0.92      1.00      0.96     56538
           1       0.94      0.07      0.13      4965

    accuracy                           0.92     61503
   macro avg       0.93      0.53      0.55     61503
weighted avg       0.93      0.92      0.89     61503



In [41]:

# Lista de tus tres diccionarios
all_test = [results, results_advanced, results_tuned]

# Consolidamos todos los experimentos en una sola lista de items
all_items = []
for d in all_test:
    all_items.extend(d.items())

# Buscamos el item que tiene el valor máximo en la clave 'auc'
best_exp, best_info = max(all_items, key=lambda x: x[1]['auc'])

print(f"Best Experiment: {best_exp}")
print(f"Best AUC: {best_info['auc']}")

Best Experiment: 29_balanced_diverse
Best AUC: 0.789015928721326


In [42]:
xgb_model = XGBClassifier(**params_29_balanced_diverse, n_jobs=-1, random_state=42, enable_categorical=True)
xgb_model.fit(
    X_train,
    y_train,
    eval_set=[(X_val, y_val)],
    verbose=True)

[0]	validation_0-auc:0.68111
[1]	validation_0-auc:0.69749
[2]	validation_0-auc:0.70622
[3]	validation_0-auc:0.71398
[4]	validation_0-auc:0.72197
[5]	validation_0-auc:0.73098
[6]	validation_0-auc:0.73252
[7]	validation_0-auc:0.73198
[8]	validation_0-auc:0.73239
[9]	validation_0-auc:0.73160
[10]	validation_0-auc:0.73471
[11]	validation_0-auc:0.73628
[12]	validation_0-auc:0.73794
[13]	validation_0-auc:0.73781
[14]	validation_0-auc:0.73809
[15]	validation_0-auc:0.73889
[16]	validation_0-auc:0.73941
[17]	validation_0-auc:0.73953
[18]	validation_0-auc:0.74033
[19]	validation_0-auc:0.73983
[20]	validation_0-auc:0.74013
[21]	validation_0-auc:0.74087
[22]	validation_0-auc:0.74136
[23]	validation_0-auc:0.74120
[24]	validation_0-auc:0.74116
[25]	validation_0-auc:0.74193
[26]	validation_0-auc:0.74254
[27]	validation_0-auc:0.74304
[28]	validation_0-auc:0.74272
[29]	validation_0-auc:0.74233
[30]	validation_0-auc:0.74230
[31]	validation_0-auc:0.74235
[32]	validation_0-auc:0.74227
[33]	validation_0-au

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=0.68, colsample_bynode=0.73,
              colsample_bytree=0.62, device='cuda', early_stopping_rounds=None,
              enable_categorical=True, eval_metric='auc', feature_types=None,
              feature_weights=None, gamma=2.7, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.011, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=1, max_depth=6,
              max_leaves=35, min_child_weight=28, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=6500,
              n_jobs=-1, num_parallel_tree=None, ...)

In [40]:
# Probabilidades (idéntico conceptualmente)
y_val_proba = xgb_model.predict_proba(X_val)[:, 1]

# AUC (esto es lo importante)
auc = roc_auc_score(y_val, y_val_proba)

#Threshold (solo para métricas de clasificación)
adj_threshold = 0.6

y_val_pred = (y_val_proba >= adj_threshold).astype(int)

# Classification report
report = classification_report(y_val, y_val_pred)

#Confusion matrix manual
TP = np.sum((y_val == 1) & (y_val_pred == 1))
FP = np.sum((y_val == 0) & (y_val_pred == 1))
TN = np.sum((y_val == 0) & (y_val_pred == 0))
FN = np.sum((y_val == 1) & (y_val_pred == 0))

FPR = FP / (FP + TN)
Precision = TP / (TP + FP)

print(f"FPR: {FPR:.4f}")
print(f"Precision (PPV): {Precision:.4f}")
print(f"AUC validación: {auc:.5f}")
print(report)

FPR: 0.0885
Precision (PPV): 0.2933
AUC validación: 0.78803
              precision    recall  f1-score   support

           0       0.95      0.91      0.93     56538
           1       0.29      0.42      0.34      4965

    accuracy                           0.87     61503
   macro avg       0.62      0.66      0.64     61503
weighted avg       0.89      0.87      0.88     61503



In [13]:
import optuna
from sklearn.metrics import roc_auc_score

def objective(trial):

    params = {
        'objective': 'binary:logistic',
        'eval_metric': 'auc',
        'tree_method': 'hist',
        'device': 'cuda',
        'booster': 'gbtree',

        # Ranges
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.03, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 3000, 8000),
        'max_depth': trial.suggest_int('max_depth', 3, 8),
        'min_child_weight': trial.suggest_int('min_child_weight', 10, 40),
        'subsample': trial.suggest_float('subsample', 0.6, 0.9),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 0.9),
        'max_delta_step': trial.suggest_float('max_delta_step', 0, 10),

        'scale_pos_weight': trial.suggest_float('scale_pos_weight', optimal_scale * 0.7, optimal_scale * 1.3),

        'reg_alpha': trial.suggest_float('reg_alpha', 0.1, 5.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.1, 5.0),
        'gamma': trial.suggest_float('gamma', 0, 5.0),
    }

    # Training
    model = XGBClassifier(**params, n_jobs=-1, random_state=42, early_stopping_rounds=100, enable_categorical=True)
    model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)

    # AUC
    preds = model.predict_proba(X_val)[:, 1]
    return roc_auc_score(y_val, preds)

# Progress callback
def callback(study, trial):
    if trial.number % 10 == 0:
        print(f"Trial {trial.number}/150: Current={trial.value:.5f}, Best={study.best_value:.5f}")

# 50 tests
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=150, callbacks=[callback] ,show_progress_bar=True)

best_params = study.best_params
print("Bests params:", best_params)

[I 2025-12-30 17:59:00,512] A new study created in memory with name: no-name-766fa674-b0f4-439a-8b0d-fbd7a13b8fc2


  0%|          | 0/150 [00:00<?, ?it/s]

[I 2025-12-30 17:59:33,055] Trial 0 finished with value: 0.7856212775572842 and parameters: {'learning_rate': 0.01935361046951764, 'n_estimators': 6496, 'max_depth': 5, 'min_child_weight': 36, 'subsample': 0.7290177802879204, 'colsample_bytree': 0.7603625941616189, 'max_delta_step': 7.9219715237227994, 'scale_pos_weight': 14.755687301045128, 'reg_alpha': 2.6451854005630784, 'reg_lambda': 2.001109196962365, 'gamma': 4.203471669981507}. Best is trial 0 with value: 0.7856212775572842.
Trial 0/150: Current=0.78562, Best=0.78562
[I 2025-12-30 18:00:08,756] Trial 1 finished with value: 0.786105084453889 and parameters: {'learning_rate': 0.02205595880707065, 'n_estimators': 3677, 'max_depth': 7, 'min_child_weight': 27, 'subsample': 0.883359570366057, 'colsample_bytree': 0.6175476897801614, 'max_delta_step': 6.167609294286889, 'scale_pos_weight': 8.712887665662887, 'reg_alpha': 3.5755612747091043, 'reg_lambda': 2.396282249488366, 'gamma': 3.4221109422425036}. Best is trial 1 with value: 0.7861

KeyboardInterrupt: 